In [1]:
import pandas as pd
import time

start = time.time()

reg_file_name = "рег.xlsx"



str_columns = ["Товар", "Название товара", "Название кластера", "ID группы закупок", 
               "ID подкатегории", "Тип акционной цены", "Акционная полочная цена (м)",
               "Имя конкурента с первой минимальной регулярной ценой (Кластер)", "Закупочная цена (м)"]


reg = pd.read_excel(reg_file_name,
                    dtype={col: str for col in str_columns},
                    skiprows=1)

reaction_step = pd.read_excel("(правила).xlsx",
                          sheet_name="5%",
                          dtype=str)
# display(reaction_step) 

reg.rename(columns={"Товар":"товар",
                    "Название товара":"наименование",
                    "Название кластера":"кластер",
                    "ID группы закупок":"гз",
                    "Регулярная полочная цена (м)":"рпц тек",
                    "Тип акционной цены":"тип апц",
                    "Маржа М1 (м) %":"м1 рпц",
                    "Акционная полочная цена T+1 (м)":"апц тек",
                    "Закупочная цена с НДС (м)":"зц",
                    "Признак ЧТМ":"признак чтм",
                    "Первая минимальная цена конкурента (Кластер)":"мин цена конкурента рпц",
                    "Имя конкурента с первой минимальной регулярной ценой (Кластер)":"имя мин конкурента рпц",
                    "Новая регулярная цена (м)":"рпц итог",
                    "Изменение регулярной цены % (м)":"откл рпц",
                    "Маржа (М1) по новой регулярной цене (м) %":"м1 рпц итог",
                    "Признак EPP":"признак epp"}, inplace=True)

def reg_price_fix(row):
    if row["признак epp"] == 1:
        row["мин цена конкурента рпц"] = row["Минимальная комбинированная цена конкурента (Кластер)"]
        row["имя мин конкурента рпц"] = row["Имя конкурента с первой минимальной комбинированной ценой (Кластер)"]
           
    return row["мин цена конкурента рпц"], row["имя мин конкурента рпц"]

reg[["мин цена конкурента рпц", "имя мин конкурента рпц"]] = reg.apply(reg_price_fix, axis=1, result_type="expand")
selected_columns = ["товар", "наименование", "кластер", "гз", "ID подкатегории", "рпц тек", "тип апц", 
                    "апц тек", "зц","мин цена конкурента рпц", "имя мин конкурента рпц",
                    "рпц итог", "признак epp", "признак чтм"]

reg = reg[selected_columns].copy()
reg["товар"] = reg["ID подкатегории"] + reg["товар"].str[-6:]
# reg["товар"] = reg["товар"].str[-6:]

reg.drop(["ID подкатегории"], axis=1, inplace=True)

float_columns = ["рпц тек", "апц тек", "зц", "мин цена конкурента рпц", "рпц итог"]
reg[float_columns] = reg[float_columns].astype(float).round(2)
    
# reg.drop(["Товар"], axis=1, inplace=True)


# "м1 рег"
# "откл рпц", "м1 рпц итог"

reg["м1 рпц тек"] = (reg["рпц тек"] - reg["зц"])/ reg["рпц тек"]
reg["м1 рпц итог"] = (reg["рпц итог"] - reg["зц"])/ reg["рпц итог"]
# reg["откл рпц"] = (reg["рпц итог"] - reg["рпц тек"])/ max(reg["рпц тек"], reg["рпц итог"])
reg["откл рпц"] = (reg["рпц итог"] - reg["рпц тек"])/ reg["рпц тек"]

reg["пкг"] = reg["товар"].str.slice(0, 6)

disc = ["КРАСНОЕ", "СЕМИШАГОФФ", "СМАРТ", "ЧИЖИК", "ДА", "ДОБРОЦЕН", "МАЯК", "НАХОДКА", 
        "ПОБЕДА", "ПОКУПАЛКО", "ПОКУПОЧКА", "СВЕТОФОР", "ХЛЕБ", "ЧЕСНОК", "БРИСТОЛЬ"]

ffc = ["P01", "P09", "P17", "P20", "P29", "P30", "P32", "P39", "P40", "P42", "P47", "P48", "P51" ]


def commenting_reg(row):
    if (pd.notna(row["тип апц"])):
        return "не меняем рпц: действует апц"
    elif (row["рпц итог"] == 0):
        return "не меняем рпц: нет изменений"
    elif (row["зц"] == 0):
        return "не меняем рпц: нет зц"

    elif (row["гз"] == "P06" and 
          abs(row["откл рпц"]) < 0.1):
        return "не меняем рпц: откл < 10%"

    elif (row["признак epp"] == 1 and 
          abs(row["откл рпц"]) < 0.05):
        return "не меняем рпц: откл < 5%"
    
    elif (row["имя мин конкурента рпц"] in disc and 
          abs(row["откл рпц"]) < 0.05):
        return "не меняем рпц: откл < 5%"
    
    elif (row["пкг"] in reaction_step["пкг"].values and
          abs(row["откл рпц"]) < 0.05):
        return "не меняем рпц: откл < 5%"
    
    elif abs(row["откл рпц"]) < 0.01:
        return "не меняем рпц: откл < 1%"  
    
    elif (row["имя мин конкурента рпц"] == "КРАСНОЕ И БЕЛОЕ" and
          row["откл рпц"] < 0):
        return "заводим апц"
    
    elif (row["гз"] in ffc and row["м1 рпц итог"] < 0 and row["откл рпц"] < 0):
        return "заводим бу"
    
    elif ((row["гз"] == "P17" or row["гз"] == "P42") and row["откл рпц"] < 0):
        return "заводим бу"
    
    elif (row["признак epp"] == 1 and row["м1 рпц итог"] < 0 and row["откл рпц"] < 0):
        return "заводим бу"
    
    else:
        return "меняем рпц"

reg["комментарий рпц"] = reg.apply(commenting_reg, axis=1)

reg.to_excel("рег отчет.xlsx",
                 index=False)
print ('создан файл - ' + str(time.time() - start))


создан файл - 3.915461540222168


In [2]:
import pandas as pd
import time

start = time.time()

promo_file_name = "промо.xlsx"



str_columns = ["Товар", "Текущая ценовая зона", "Название товара", "Название кластера", "ID группы закупок", 
               "Акционная полочная цена (м)", "Тип акционной цены", "Закупочная цена (м)", 
               "Минимальная комбинированная цена конкурента (Кластер)", "Новая промо цена (м) ", 
               "Изменение промо цены % (м)","ID подкатегории",
               "Имя конкурента с первой минимальной комбинированной ценой (Кластер)"]


promo = pd.read_excel(promo_file_name,
                      dtype={col: str for col in str_columns},
                      skiprows=1)


promo.rename(columns={"Товар":"товар",
                      "Текущая ценовая зона":"тк",
                      "Название товара":"наименование",
                      "Название кластера":"кластер",
                      "ID группы закупок":"гз",
                      "Акционная полочная цена T+1 (м)":"апц тек",
                      "Тип акционной цены":"тип апц",
                      "Закупочная цена с НДС (м)":"зц",
                      "Признак ЧТМ":"признак чтм",
                      "Минимальная комбинированная цена конкурента (Кластер)":"мин цена конкурента апц",
                      "Новая промо цена (м) ":"апц итог",
                      "Изменение промо цены % (м)":"откл апц",
                      "Имя конкурента с первой минимальной комбинированной ценой (Кластер)":"имя мин конкурента апц",
                      "Признак EPP":"признак epp",
                      "ID акции":"код ра"}, inplace=True)

selected_columns = ["товар", "наименование", "тк", "кластер", "гз", "апц тек", "тип апц", "зц", "мин цена конкурента апц", 
                    "апц итог","ID подкатегории", "код ра", "признак epp", "имя мин конкурента апц", "признак чтм"]

promo = promo[selected_columns].copy()

# promo["товар"] = promo["ID подкатегории"] + promo["товар"].str[-6:]
# promo["товар"] = promo["товар"].str[-6:]
promo["товар"] = promo["ID подкатегории"] + promo["товар"].str[-6:]

promo.drop(["ID подкатегории"], axis=1, inplace=True)

float_columns = ["апц тек", "зц", "мин цена конкурента апц", "апц итог"]
promo[float_columns] = promo[float_columns].astype(float).round(2)
    

promo["м1 апц тек"] = (promo["апц тек"] - promo["зц"])/ promo["апц тек"]
promo["м1 апц итог"] = (promo["апц итог"] - promo["зц"])/ promo["апц итог"]
promo["откл апц"] = (promo["апц итог"] - promo["апц тек"])/ promo["апц тек"]

promo["пкг"] = promo["товар"].str.slice(0, 6)

disc = ["КРАСНОЕ", "СЕМИШАГОФФ", "СМАРТ", "ЧИЖИК", "ДА", "ДОБРОЦЕН", "МАРИЯ-РА", "МАЯК", "МОНЕТКА", "НАХОДКА", 
        "ПОБЕДА", "ПОКУПАЛКО", "ПОКУПОЧКА", "СВЕТОФОР", "ХЛЕБ", "ЧЕСНОК", "ЯРЧЕ", "БРИСТОЛЬ"]

def commenting_promo(row):
    if not pd.notna(row["тип апц"]):
        return "не меняем апц: действует рпц"
    elif (row["апц итог"] == 0):
        return "не меняем апц: нет изменеий"
    
    elif (row["признак epp"] == 1 and 
          abs(row["откл апц"]) < 0.05):
        return "не меняем апц: откл < 5%"
    
    elif (row["имя мин конкурента апц"] in disc and 
          abs(row["откл апц"]) < 0.05):
        return "не меняем апц: откл < 5%"
    
    elif (row["пкг"] in reaction_step["пкг"].values and
          abs(row["откл апц"]) < 0.05):
        return "не меняем апц: откл < 5%"
    
    elif abs(row["откл апц"]) < 0.01:
        return "не меняем апц: откл < 1%" 
    
    elif ((row["гз"] == "P12" or row["гз"] == "P38") and row["откл апц"] < 0 
          and row["признак epp"] == 0):
        return "заводим бу"
    
    else:
        return "меняем апц"

promo["комментарий апц"] = promo.apply(commenting_promo, axis=1)

promo.to_excel("промо отчет.xlsx",
                 index=False)
print ('создан файл - ' + str(time.time() - start))


создан файл - 1.599757432937622


In [3]:
import pandas as pd
import time

start = time.time()


str_columns = ["товар", "наименование", "кластер", "тк"]

reg_file_name = "рег отчет.xlsx"
reg = pd.read_excel(reg_file_name,
                    dtype={col: str for col in str_columns})

promo_file_name = "промо отчет.xlsx"
promo = pd.read_excel(promo_file_name,
                    dtype={col: str for col in str_columns})

pf = pd.read_excel("(правила).xlsx",
                   sheet_name="цс",
                   dtype="str")

tk = pd.read_excel("(правила).xlsx",
                   sheet_name="тк",
                   dtype="str")

tk = pd.read_excel("(правила).xlsx",
                   sheet_name="тк",
                   dtype="str")
lp = pd.read_excel("(правила).xlsx",
                   sheet_name="связное цо",
                   dtype="str")

reg = reg.merge(tk[["кластер", "тк", "цо тк", "со", "номер кластера"]], how='left', left_on="кластер", right_on="кластер")

reg["проверка по цо тк"] = reg.apply(lambda row: "да" if row["тк"] == row["цо тк"] 
                                       else "нет", axis=1)


promo = promo.merge(tk[["тк", "цо тк", "со", "номер кластера"]], how='left', left_on="тк", right_on="тк")
promo["проверка по цо тк"] = promo.apply(lambda row: "да" if row["тк"] == row["цо тк"] 
                                       else "нет", axis=1)


union = reg.merge(promo, how='outer', left_on=["товар", "тк","наименование"], right_on=["товар", "тк","наименование"])

def union_cols(col):
    
    union[col] = union[col+"_y"].fillna(union[col+"_x"])
    union.drop([col+"_x", col+"_y"], axis=1, inplace=True)

union_cols("кластер")
union_cols("гз")
union_cols("апц тек")
union_cols("зц")
union_cols("со")
union_cols("пкг")
union_cols("признак чтм")
union_cols("номер кластера")
union_cols("тип апц")
union_cols("признак epp")
union_cols("цо тк")
union_cols("проверка по цо тк")

union = union.merge(pf[["товар","цс"]], how='left', left_on="товар", right_on="товар")

union = union.merge(pf[["товар","цс", "наименование"]], how='left', left_on="цс", right_on="цс")

# display(union)

union["товар_y"].fillna(union["товар_x"], inplace=True)
union["есть в мониторинге?"] = union.apply(lambda row: "да" if row["товар_x"] == row["товар_y"] 
                                       else "нет - расценяется по цс", axis=1)
# display(union)
union["товар из мониторинга"] = union["товар_x"]
union["расценяемый товар"] = union["товар_y"]
union.drop(["товар_y", "товар_x"], axis=1, inplace=True)

# union["наименование_y"].fillna(union["наименование_x"], inplace=True)
union["наименование_y"].fillna(union["наименование_x"], inplace=True)
union["наименование товара из мониторинга"] = union["наименование_x"]
union["наименование расценяемого товара"] = union["наименование_y"]

# union.insert(1, "наименование",union["наименование_y"])
union.drop(["наименование_y", "наименование_x"], axis=1, inplace=True)


union = union.merge(lp, how='left', left_on="товар из мониторинга", right_on="основной товар")



def move_cols(row):
    if pd.notna(row["связанный товар"]):
        return row["связанный товар"], row["наименование"]
    else:
        return row["расценяемый товар"], row["наименование расценяемого товара"]
    
    
union[["расценяемый товар", "наименование расценяемого товара"]] = union.apply(move_cols, axis=1, result_type="expand")    
    


def linked_pricing(row):
    if pd.notna(row["связка"]) and str(row["коэффициент"])!= "1":
        
        
        rp = row["рпц итог"] * float(row["коэффициент"])
        if rp == 0: 
            rp = 0
        elif rp < 50:
            rp = rp // 1 + 0.99
        else:
            rp = rp // 5 * 5 + 4.99
        pp = None
        
        return "нет - расценяется по связному цо", None, None, None, rp, None , None, None, None, None, None, pp, None
    else:
        return (row["есть в мониторинге?"], row["зц"], row["рпц тек"], row["м1 рпц тек"], row["рпц итог"],
                row["м1 рпц итог"], row["откл рпц"], row["апц тек"], row["тип апц"], row["код ра"], 
                row["м1 апц тек"], row["апц итог"], row["откл апц"])
       
    
union[["есть в мониторинге?", "зц", "рпц тек", "м1 рпц тек", "рпц итог", 
      "м1 рпц итог", "откл рпц", "апц тек", "тип апц", "код ра", 
      "м1 апц тек", "апц итог", "откл апц"]] = union.apply(linked_pricing, axis=1, result_type="expand")




union["комменатрий КС 1 (для изменения рпц, заведения бу, заведения апц)"]= [None] * len(union)
union["комменатрий КС 2 (для изменения апц)"]= [None] * len(union)


cols_order = ["товар из мониторинга", "наименование товара из мониторинга", "расценяемый товар",
              "наименование расценяемого товара", "цс", "есть в мониторинге?", "гз", "пкг", "со", "кластер", 
              "номер кластера","тк", "цо тк", "проверка по цо тк", "признак epp", "признак чтм", "зц", 
              "рпц тек", "м1 рпц тек","рпц итог", "м1 рпц итог", "откл рпц", "мин цена конкурента рпц",
              "имя мин конкурента рпц","комментарий рпц", 
              "комменатрий КС 1 (для изменения рпц, заведения бу, заведения апц)", 
              "апц тек", "тип апц", "код ра", "м1 апц тек", "апц итог", "м1 апц итог", "откл апц",
              "мин цена конкурента апц", "имя мин конкурента апц", "комментарий апц", 
              "комменатрий КС 2 (для изменения апц)"
             ]

union2 = union[cols_order]

union2.to_excel("результат мониторинга.xlsx",
                 index=False)
print ('создан файл - ' + str(time.time() - start))

создан файл - 10.352175951004028
